In [131]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.neighbors import NearestNeighbors

In [ ]:
import os 
nueva_ruta = r'C:\Users\Luis\Documents\GitHub\TFM_KSCHOOL\Luis\PreModelado_TFM_Palas\Noviembre\PreModelado_Palas'
os.chdir(nueva_ruta)
print("Nueva ruta actual:", os.getcwd())


In [133]:
df_form3 = pd.read_csv('df_scaled_formularios_3.0.csv')
df_palas3_sin_filtrar = pd.read_csv('df_scaled_palas_3.0.csv')

In [ ]:

# Seleccionar una fila aleatoria de df_form
random_row = df_form3.sample(1)

# Extraer las coordenadas X e Y de la fila aleatoria
x_random = random_row['Score_Escalar_Lesion'].values[0]
y_random = random_row['Score_Escalar_Nivel'].values[0]

# Extraer el sexo del formulario
sexo_form = random_row["Indique su sexo"].values[0]  # Usar .values[0] para claridad

# Filtrar filas en base al género indicado
if sexo_form == "Hombre":
    # Incluir jugadores "hombre" o "mixta"
    df_palas3 = df_palas3_sin_filtrar[df_palas3_sin_filtrar["Jugador"].isin(["hombre", "mixta"])]
elif sexo_form == "Mujer":
    # Incluir jugadores "mujer" o "mixta"
    df_palas3 = df_palas3_sin_filtrar[df_palas3_sin_filtrar["Jugador"].isin(["mujer", "mixta"])]

# Verificar el resultado del filtrado
print(f"Sexo del formulario: {sexo_form}")
print(f"Cantidad de palas después del filtrado: {len(df_palas3)}")

In [ ]:

# Crear figura
fig = plt.figure(figsize=(20, 14))

# Crear un gráfico en 2D
ax = fig.add_subplot(111)

# Definir los valores de cada eje para el DataFrame principal (df_palas)
x = df_palas3['score_lesion_ajustado']
y = df_palas3['score_nivel_ajustado']

# Crear el gráfico de dispersión 2D para df_palas
ax.scatter(x, y, color='blue', label='Datos de Palas')

# Crear los puntos adicionales
ax.scatter([x_random], [y_random],
           color='red', s=100, label='Formularios')

# Calcular los límites con buffer
x_min, x_max = x.min(), x.max()
y_min, y_max = y.min(), y.max()
buffer = 0.01  # Cambia este valor para ajustar el tamaño del margen

# Ajustar límites de los ejes con el buffer
ax.set_xlim(x_min - buffer, x_max + buffer)
ax.set_ylim(y_min - buffer, y_max + buffer)

# Añadir etiquetas a los ejes
ax.set_xlabel('Score de Lesión (Escalado)')
ax.set_ylabel('Score de Nivel (Escalado)')

# Título del gráfico
ax.set_title('Gráfico 2D de Score de Lesión, Score de Nivel y Formulario 3 horquillas')

# Añadir leyenda para diferenciar los conjuntos de puntos
ax.legend()

# Configurar las particiones de Y para solo mostrar en 0.45, 0.55, 0.65
ax.set_yticks([0.45, 0.55, 0.65])  # Solo las marcas deseadas en Y
ax.grid(True, which='both', axis='both', linestyle='--', color='gray', alpha=0.5)

# Mostrar el gráfico
plt.show()

In [136]:
label_mapping = {
    "Balance": {"No data": 0, "bajo": 1, "medio": 2, "alto": 3},
    "Nucleo": {"No data": 0, "foam": 1, "medium eva": 2, "hard eva": 3, "soft eva": 4},
    "Cara": {"No data": 0, "fibra de vidrio": 1, "mix": 2, "fibra de carbono": 3},
    "Dureza": {"No data": 0, "blanda": 1, "media": 2, "dura": 3},
    "Nivel de Juego": {"No data": 0, "principiante": 1, "avanzado": 2, "pro": 3},
    "Forma": {"No data": 0, "redonda": 1, "lágrima": 2, "diamante": 3},
    "Superficie": {"No data": 0, "lisa": 1, "rugosa": 2},
    "Tipo de Juego": {"No data": 0, "control": 1, "polivalente": 2, "potencia": 3},
}

# Función para invertir un diccionario
def invertir_label_mapping(label_mapping):
    inverted_mapping = {}
    for key, value_dict in label_mapping.items():
        inverted_mapping[key] = {v: k for k, v in value_dict.items()}
    return inverted_mapping

# Invertir el diccionario
inverted_label_mapping = invertir_label_mapping(label_mapping)

In [137]:
def invertir_label_mapping(label_mapping):
    return {key: {v: k for k, v in value_dict.items()} for key, value_dict in label_mapping.items()}

# Invertir el diccionario de mapeo
inverted_label_mapping = invertir_label_mapping(label_mapping)

In [ ]:

# Función para obtener las palas recomendadas usando KNN
def encontrar_vecinos_mas_cercanos_knn_2d(df_palas3, x_random, y_random, considerar_precio, precio_maximo):
    # Características para el modelo KNN
    knn_features = ['score_lesion', 'score_nivel']
    
    if considerar_precio:
        knn_features.append('Precio')
        df_palas3 = df_palas3[df_palas3['Precio'] <= precio_maximo]
    
    knn = NearestNeighbors(n_neighbors=3)
    knn.fit(df_palas3[knn_features])
    
    # Crear un punto de referencia para KNN
    reference_point = pd.DataFrame(
        [[x_random, y_random] + ([precio_maximo] if considerar_precio else [])],
        columns=knn_features
    )
    
    distances, indices = knn.kneighbors(reference_point)
    
    palas_recomendadas = df_palas3.iloc[indices[0]].copy()

    # Optimización: mapear las columnas en un solo paso
    for column in ['Nivel de Juego', 'Tipo de Juego', 'Balance', 'Nucleo', 'Cara', 'Dureza', 'Forma', 'Superficie']:
        palas_recomendadas[column] = palas_recomendadas[column].map(inverted_label_mapping[column])

    # Devolver las palas recomendadas con la información más completa
    return palas_recomendadas[['Palas', 'Nivel de Juego', 'Tipo de Juego', 'Balance', 'Nucleo', 'Cara', 'Dureza', 'Forma', 'Superficie', 'Precio', 'score_lesion_ajustado', 'score_nivel_ajustado']]


# Llamar a la función para obtener las palas recomendadas usando KNN
recomendaciones_knn = encontrar_vecinos_mas_cercanos_knn_2d(df_palas3, x_random, y_random, considerar_precio=False, precio_maximo=100)

# Mostrar las recomendaciones de KNN
print("Palas recomendadas usando KNN:")
print(recomendaciones_knn)


In [ ]:
# Función para obtener las palas dentro del cuadrante con información completa
def obtener_palas_por_cuadrante(df_palas3, x_random, y_random):
    # Particiones de X y Y
    particiones_x = np.arange(0.2, 1.0, 0.1)  # 0.2, 0.3, ..., 0.9
    particiones_y = [0.2, 0.45, 0.55, 0.65, 0.9]
    
    # Determinar el cuadrante de X
    x_cuadrante = next(((particiones_x[i], particiones_x[i + 1]) 
                        for i in range(len(particiones_x) - 1) 
                        if particiones_x[i] <= x_random < particiones_x[i + 1]), 
                       (particiones_x[-2], particiones_x[-1]))  # Último intervalo

    # Determinar el cuadrante de Y
    y_cuadrante, y_cuadrante_upper = next(((particiones_y[i], particiones_y[i + 1]) 
                                            for i in range(len(particiones_y) - 1) 
                                            if particiones_y[i] <= y_random < particiones_y[i + 1]), 
                                           (particiones_y[-1], particiones_y[-1]))  # Último valor

    print(f"Cuadrante X: {x_cuadrante}, Cuadrante Y: ({y_cuadrante}, {y_cuadrante_upper})")

    # Filtrar las palas dentro de este cuadrante
    df_palas_filtrado = df_palas3[(df_palas3['score_lesion_ajustado'] >= x_cuadrante[0]) & 
                                  (df_palas3['score_lesion_ajustado'] < x_cuadrante[1]) & 
                                  (df_palas3['score_nivel_ajustado'] >= y_cuadrante) & 
                                  (df_palas3['score_nivel_ajustado'] < y_cuadrante_upper)& 
                                (~df_palas3['Palas'].isin(recomendaciones_knn['Palas']))].copy()

    # Si hay palas filtradas, mapear las etiquetas
    if not df_palas_filtrado.empty:
        # Optimización: mapear las columnas en un solo paso
        for column in ['Nivel de Juego', 'Tipo de Juego', 'Balance', 'Nucleo', 'Cara', 'Dureza', 'Forma', 'Superficie']:
            df_palas_filtrado[column] = df_palas_filtrado[column].map(inverted_label_mapping[column])

        # Agregar las columnas adicionales de interés
        return df_palas_filtrado[['Palas', 'Nivel de Juego', 'Tipo de Juego', 'Balance', 'Nucleo', 'Cara', 'Dureza', 
                                  'Forma', 'Superficie', 'Precio', 'score_lesion_ajustado', 'score_nivel_ajustado']]

    else:
        # Retornar un DataFrame vacío si no hay resultados
        return pd.DataFrame(columns=['Palas', 'Nivel de Juego', 'Tipo de Juego', 'Balance', 'Nucleo', 'Cara', 'Dureza', 
                                     'Forma', 'Superficie', 'Precio', 'score_lesion_ajustado', 'score_nivel_ajustado'])



# Llamar a la función para obtener las palas recomendadas dentro del cuadrante
palas_recomendadas_rejilla = obtener_palas_por_cuadrante(df_palas3, x_random, y_random)

# Mostrar las recomendaciones de palas en el cuadrante
print("Palas recomendadas en el cuadrante del punto aleatorio:")
print(palas_recomendadas_rejilla)

In [ ]:
if len(palas_recomendadas_rejilla) < 2:
    dos_palas_recomendadas_rejilla = palas_recomendadas_rejilla
    
else:
    dos_palas_recomendadas_rejilla = palas_recomendadas_rejilla.sample(n=2, replace=False)



# Si 'recomendaciones' y 'dos_palas_recomendadas_rejilla' son DataFrames
palas_definitivas = pd.concat([recomendaciones_knn, dos_palas_recomendadas_rejilla], ignore_index=True)

# Mostrar el resultado
print(palas_definitivas)